# 프롬프트 엔지니어링을 위한 모범 사례

## 주요 용어

프롬프트 엔지니어링: AI 모델이 원하는 출력을 생성하도록 입력을 설계하고 수정하는 실습.

토큰화: 텍스트를 모델이 이해하고 처리할 수 있는 작은 단위인 토큰으로 변환하는 과정.

지시 조정된 LLM(Instruction-Tuned LLMs): 특정 지시를 통해 응답 정확성과 관련성을 개선하기 위해 세부 조정된 대규모 언어 모델(LLM).

출처: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api


## 왜 프롬프트 엔지니어링이 필요한가?

_왜_ 프롬프트 엔지니어링이 필요한지 이야기해 봅시다. 그 이유는 현재 LLM이 _신뢰할 수 있고 일관된 완성_ 을 달성하기 어렵게 만드는 여러 과제를 제시하기 때문입니다. 프롬프트 작성 및 최적화에 노력을 기울이지 않으면 이러한 과제를 해결하기 어렵습니다. 예를 들어:

1. **모델 응답은 확률적입니다.** _동일한 프롬프트_ 는 다른 모델이나 모델 버전에서 다른 응답을 생성할 가능성이 높습니다. 그리고 _동일한 모델_ 에서도 다른 시간에 다른 결과를 생성할 수 있습니다. _프롬프트 엔지니어링 기술은 더 나은 가드레일을 제공하여 이러한 변동을 최소화하는 데 도움을 줄 수 있습니다_.
- 답변을 JSON으로만 출력해.
- 키 이름은 name, age, job만 사용해.
- 그 외 문장은 출력하지 마.

2. **모델은 응답을 조작할 수 있습니다.** 모델은 _크지만 유한한_ 데이터셋으로 사전 훈련되었기 때문에 해당 훈련 범위를 벗어난 개념에 대한 지식이 부족합니다. 결과적으로 부정확하거나 상상적이거나 알려진 사실과 직접적으로 모순되는 완성을 생성할 수 있습니다. _프롬프트 엔지니어링 기술은 사용자가 AI에게 인용이나 추론을 요청하여 이러한 조작을 식별하고 완화하는 데 도움을 줄 수 있습니다_.
- 너가 확실하지 않은 내용은 "모르겠다"고 말해.
- 출력에는 반드시 출처나 이유를 포함해.
- 단계별로 생각하고 마지막에 근거를 써줘.


3. **모델의 능력은 다양합니다.** 최신 모델이나 모델 세대는 더 풍부한 기능을 제공하지만 비용 및 복잡성에서 고유한 특성과 트레이드오프를 가져옵니다. _프롬프트 엔지니어링은 모델별 요구 사항에 맞게 차이를 추상화하고 확장 가능하고 원활한 방식으로 적응할 수 있는 모범 사례와 워크플로를 개발하는 데 도움을 줄 수 있습니다_.
- 추론력 강화 → 단계적 사고 강제
- 지시 실행력 강화 → 출력 형식을 매우 상세히 정의
- 사실 기반 응답 강화 → 근거 제공 강제


OpenAI 또는 Azure OpenAI Playground에서 이를 직접 확인해 보세요:

- 동일한 프롬프트를 다른 LLM 배포(예: OpenAI, Azure OpenAI, Hugging Face)에서 사용해 보세요 - 변동을 확인했나요?
- 동일한 프롬프트를 _동일한_ LLM 배포(예: Azure OpenAI Playground)에서 반복적으로 사용해 보세요 - 이러한 변동은 어떻게 달랐나요?

In [3]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-12-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

# 1. 최신 모델 사용

최고의 결과를 위해 최신 모델을 사용하세요.

# 2. 프롬프트의 시작 부분에 지침을 배치하고 ### 또는 """로 지침과 컨텍스트를 구분하세요

In [2]:
response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages = [{"role":"system", "content":"당신은 도움이 되는 어시스턴트입니다."},
                                {"role":"user","content": '아래 텍스트를 가장 중요한 요점의 글머리표 목록으로 한글로 요약해 주세요. \n\n \
                ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
                capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
                raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
                broadly sharing benefits and the need to prioritize safety. \
                Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###',}],
                max_tokens=400,)

print(response.choices[0].message.content)

- OpenAI와 Microsoft가 파트너십을 연장하기로 발표함
- Microsoft가 수년간 수십억 달러를 추가로 투자함 (2019, 2021년에 이어 다년간 투자)
- 이번 투자는 OpenAI가 독립적인 연구를 지속하고 더 안전하고 유용하며 강력한 AI 개발을 지원
- OpenAI는 capped-profit 구조로 운영되며, 비영리 단체에 의해 관리됨
- 이러한 구조는 사명 수행을 위한 자본 조달과 이익 공유, 안전 우선이라는 핵심 가치를 유지하는 데 도움 됨
- Microsoft도 OpenAI의 비전과 가치를 공유하며, 파트너십이 진전에 중요한 역할을 함


# 3. 원하는 컨텍스트, 결과, 길이, 형식, 스타일 등에 대해 구체적이고 상세하게 작성하세요

In [4]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '경복궁을 위한 시 한편 써줘',}],
        max_tokens=400,)

print(response.choices[0].message.content)


경복궁 그늘 아래 서서

고요히 흘러가는 시간의 강,
붉은 기와와 푸른 하늘 맞닿은 자리,
경복궁이 품은 옛 이야기들,
바람결에 속삭이네, 역사의 숨결.

단청 물들인 기둥 사이로,
세월이 짙은 그림자를 드리우고,
왕과 신하의 꿈과 희망이 교차하는,
그 신비로운 공간에 내가 서 있네.

꽃 잎 한 장에도 깃든 정신,
돌담 위에 새긴 백 년의 맹세,
오늘도 변함없이 빛나는 그 이름,
경복궁, 우리의 마음에 새겨진 보물.

시간이 지나도 잊지 않을게,
그 자리에서 우리를 지키는 그대,
경복궁, 영원한 역사와 문화의 집.


In [6]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '경복궁을 위한 시 한편 써줘 \
                특히 봄에서 여름으로 가는 게절에 아름다운 풍경을 충분히 묘사해줘',}],
        max_tokens=400,)

print(response.choices[0].message.content)



경복궁 봄에서 여름으로

연둣빛 기운 가득한 봄날이여,  
고즈넉한 경복궁 담장 너머로  
살랑이는 바람이 꽃망울을 흔들고,  
연분홍 진달래빛이 고궁을 물들이네.  

고즈넉한 기와지붕 위로 햇살이 춤추고,  
꽃잎 사이로 새들의 노래가 퍼져가니,  
한지 창호 너머로 스미는 따스한 빛,  
옛 선비의 숨결이 봄바람에 실려 오다.  

이내 여름이 성큼 다가와,  
푸른 연잎은 두터운 그늘 되어 주고,  
연못 위에 비친 하늘은 더 짙푸르게 변해,  
고궁의 고요함 속에 삶이 깊어가네.  

연꽃 향기 은은히 번지는 여름,  
잔잔한 바람결에 수복이글 지고,  
봄의 꽃마음과 여름의 싱그러움이  
경복궁 한켠에 영원히 머물리라.


# 4. 원하는 출력 형식을 예제를 통해 명확히 표현하세요 (예제 1, 예제 2).

In [7]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '아래 텍스트에서 회사명 다음 연도를 추출하고 각 엔티티의 시작 인덱스와 끝 인덱스를 출력합니다.\
                Generate output as {"text": "OpenAI", "start": 28, "end": 34} \
                ###\
                OpenAI와 Microsoft가 파트너십을 연장한다는 기쁜 소식을 전하게 되어 기쁩니다. \
                Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년에 이루어진 이전 투자에 이은 것입니다. \
                2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 계속하고 더욱 안전하고 유용하며 강력한 \
                더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다. \n\n \
                ###\
                ',}],
                
        max_tokens=400,)

print(response.choices[0].message.content)



[
  {"text": "Microsoft", "start": 8, "end": 17},
  {"text": "2019년", "start": 63, "end": 67},
  {"text": "2021년", "start": 70, "end": 74}
]


In [8]:
prompt = """
                아래 텍스트에 언급된 엔티티를 추출합니다. 
                아래 텍스트에 언급된 중요한 엔티티를 추출합니다. 
                먼저 모든 회사 이름을 추출한 다음 모든 연도를 추출합니다, 
                그런 다음 콘텐츠에 맞는 특정 주제를 추출하고 마지막으로 일반적인 주요 주제를 추출합니다.\n\n 
                Desired format: 
                Company names: <comma_separated_list_of_company_names> 
                Years: -||- 
                Specific topics: -||- 
                General themes: -||- 

                ###\n\n
                OpenAI와 Microsoft가 파트너십을 연장한다는 기쁜 소식을 알려드리게 되어 기쁩니다.
                Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년 투자에 이어 
                2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 지속하고 더욱 안전하고 유용하며 강력한 
                더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다. \n\n

        """

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt}],
        max_tokens=400,)

print(response.choices[0].message.content)



Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: AI 개발, 다년간 투자, 파트너십 연장, 독립적인 연구  
General themes: 인공지능, 투자, 기술 협력


# 5. 제로샷(zero-shot)으로 시작하고, 이후 몇 가지 예제를 제공하세요. 둘 다 효과가 없으면 미세 조정을 시도하세요 (업데이트 필요)

## 제로샷(Zero-shot) → 퓨샷(Few-shot) → 미세 조정(Fine-tuning)

![Deploy](image/Deploy.png)

In [5]:
prompt = """
        OpenAI와 Microsoft가 파트너십을 연장한다는 기쁜 소식을 알려드리게 되어 기쁩니다.
        Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년 투자에 이어 
        2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 지속하고 더욱 안전하고 유용하며 강력한 
        더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다.

"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, 파트너십 연장, 다년간 투자, 수십억 달러, 2019년 투자, 2021년 투자, 독립적 연구, 안전한 AI, 유용한 AI, 강력한 AI, 개발


In [6]:
system_prompt = """

        당신은 유용한 조수입니다. 아래 해당 텍스트에서 키워드를 추출하세요..
        텍스트: 스트라이프는 웹 개발자가 웹 사이트와 모바일 애플리케이션에 결제 처리를 통합하는 데 사용할 수 있는 API를 제공합니다. 
        키워드: 스트라이프, 결제 처리, API, 웹 개발자, 웹사이트, 모바일 애플리케이션 
        ###
        텍스트: OpenAI는 텍스트를 이해하고 텍스트를 생성하는 데 매우 능숙한 최첨단 언어 모델을 학습시켰습니다.
        이해하고 텍스트를 생성하는 데 매우 능숙한 최첨단 언어 모델을 학습시켰습니다. API를 통해 이러한 모델에 액세스할 수 있으며, 언어 처리와 관련된 거의 모든 작업을
        언어 처리와 관련된 모든 작업을 해결하는 데 사용할 수 있습니다.
        키워드: 언어 모델, 텍스트 처리, API.
        
"""

user_prompt = """
        텍스트: OpenAI와 Microsoft가 파트너십을 연장한다는 소식을 전하게 되어 기쁘게 생각합니다.
        Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년에 이루어진 이전 투자에 이은 것입니다.
        2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 지속하고 더욱 안전하고 유용하며 강력한 
        더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다. 
        키워드:
"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content": system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



키워드: OpenAI, Microsoft, 파트너십, 투자, AI 개발, 안전성, 유용성, 독립적 연구


# 6. 모호하고 부정확한 설명을 줄이세요

In [11]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '새 제품에 대한 설명을 작성하세요. 이 제품은 차세대 카시트입니다. 이 제품에 대한 설명은 몇 문장으로만 짧게 작성하고 너무 길지 않아야 합니다.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



차세대 카시트는 안전성과 편안함을 한층 강화한 혁신적인 제품입니다. 인체공학적 디자인과 최신 기술로 아이의 보호와 편의를 동시에 제공합니다. 쉽게 설치할 수 있어 부모님의 부담을 덜어줍니다.


In [12]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '새 제품에 대한 설명을 작성하세요. 이 제품은 차세대 카시트입니다. 3~5문장으로 구성된 단락을 사용하여 이 제품을 설명하세요.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



이 차세대 카시트는 아이의 안전과 편안함을 최우선으로 설계되었습니다. 인체공학적 디자인과 첨단 충격 흡수 소재를 적용하여 사고 시에도 뛰어난 보호 기능을 제공합니다. 또한, 간편한 설치 시스템과 조절 가능한 기능으로 성장하는 아이에게 맞춤형 사용이 가능합니다. 세련된 디자인과 다양한 색상 옵션으로 차량 내부 인테리어와도 완벽하게 어울립니다. 이 제품은 부모님의 마음까지 안심시켜 줄 최첨단 카시트입니다.


# 7. 하지 말아야 할 것을 말하는 대신, 해야 할 것을 명확히 설명하세요

In [13]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. Customer: I can’t log in to my account.\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I’m sorry to hear that you’re having trouble logging in. Could you please tell me what error message you’re seeing or what happens when you try to log in?


In [14]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. \
                The agent will attempt to diagnose the problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. To help resolve this issue, please make sure you are entering your login details correctly and that your internet connection is stable. If the problem persists, you can find detailed troubleshooting steps and assistance in our help article here: www.samplewebsite.com/help/faq. Let me know if there's anything else I can assist you with!


# 8. 코드 생성 관련 - 특정 패턴으로 모델을 유도하기 위해 "선도 단어"를 사용하세요

In [15]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# 다음과 같은 간단한 파이썬 함수를 작성합니다. \n\
                # 1. 마일 단위로 숫자를 요청하세요.\n\
                # 2. 마일을 킬로미터로 변환합니다.',}],
        max_tokens=400,)

print(response.choices[0].message.content)


아래는 마일 단위의 숫자를 입력받아 킬로미터로 변환하는 간단한 파이썬 함수 예제입니다:

```python
def miles_to_kilometers():
    miles = float(input("마일 단위의 숫자를 입력하세요: "))
    kilometers = miles * 1.60934
    print(f"{miles} 마일은 {kilometers:.2f} 킬로미터입니다.")

# 함수 실행
miles_to_kilometers()
```

설명:
- `input()` 함수를 이용해 사용자로부터 마일 수를 입력받습니다.
- 입력 받은 값을 실수형(`float`)으로 변환합니다.
- 1마일 = 1.60934킬로미터 이므로 이를 곱해 변환합니다.
- 결과를 소수점 둘째 자리까지 출력합니다.
